In [1]:
import matplotlib.pyplot as plt

In [12]:
decisionNode = dict(boxstyle = 'sawtooth',fc = '0.8')  #决策分支节点样式
leafNode = dict(boxstyle = 'round4',fc = '0.8') #叶子节点样式
arrow_args = dict(arrowstyle = '<-')  #定义决策箭头样式

In [3]:
#centerpt是箭头终止位置(箭头的尖尖画在终止位置上)，也是对应结点和结点文本的位置。parentpt表示箭头开始的位置
def plotnode(nodetext,centerpt,parentpt,nodetype): #绘制箭头与节点文本，后三参数表示文本位置(箭头终止位置)，箭头起始位置以及箭头指向的节点类型
    createplot.ax1.annotate(nodetext,xy = parentpt,xycoords = 'axes fraction',xytext = centerpt,textcoords = 'axes fraction',
                       va = 'center',ha = 'center',bbox = nodetype,arrowprops = arrow_args)#annotate是一个注解工具

In [4]:
def getnumleaf(tree):
    numleaf = 0
    firststr = list(tree.keys())[0]
    secondict = tree[firststr]
    for key in secondict.keys():
        if type(secondict[key]).__name__ == 'dict':
            numleaf += getnumleaf(secondict[key])
        else:
            numleaf += 1
    return numleaf

In [5]:
def getdepth(tree):
    maxdepth = 0
    firststr = list(tree.keys())[0]
    secondict = tree[firststr]
    for key in secondict.keys():
        if type(secondict[key]).__name__ == 'dict':
            tempdepth = 1 + getdepth(secondict[key])
        else:
            tempdepth = 1
        if tempdepth>maxdepth:
            maxdepth = tempdepth
    return maxdepth

In [6]:
def plotmidtext(centerpt,parentpt,text):  #在箭头中填充文本
    xmid = (parentpt[0]-centerpt[0])/2.0 + centerpt[0]
    ymid = (parentpt[1]-centerpt[1])/2.0 + centerpt[1]
    createplot.ax1.text(xmid,ymid,text)

In [7]:
def plottree(tree,parentpt,nodetext):
    numleaf = getnumleaf(tree)
    depth = getdepth(tree)
    firststr = list(tree.keys())[0]
    centerpt = (plottree.xOff + (1.0 + float(numleaf))/2.0/plottree.totalW,plottree.yOff)#文本坐标(箭头终止位置)
    plotmidtext(centerpt,parentpt,nodetext)  #绘制箭头中文本  
    plotnode(firststr,centerpt,parentpt,decisionNode) #绘制箭头与分支节点文本
    #是先画箭头中文本，再画箭头和指向结点的(箭头+一个被指向结点(在箭头下面))
    #第一次画时，起始位置和终止位置一样，且输入文本为空，所以plotmidtext没有填充箭头文本
    #第一次画时。起始位置和终止位置一样，所以plotnode只画了一个结点(在终止位置处)，没有画箭头
    #下一次画时，将先在中间填充箭头文本，再画出箭头，再在箭头结束位置画出一个结点
    secondict = tree[firststr]
    plottree.yOff = plottree.yOff - 1.0/plottree.totalD
    for key in secondict.keys():
        if type(secondict[key]).__name__ == 'dict':
            plottree(secondict[key],centerpt,str(key))
        else:
            plottree.xOff = plottree.xOff + 1.0/plottree.totalW
            plotnode(secondict[key],(plottree.xOff,plottree.yOff),centerpt,leafNode) #绘制叶子节点以及其文本
            plotmidtext((plottree.xOff,plottree.yOff),centerpt,str(key)) #填充箭头中文本
    plottree.yOff = plottree.yOff + 1.0/plottree.totalD

In [8]:
def createplot(tree):
    fig = plt.figure(1,facecolor = 'white')
    fig.clf() #清除当前图像(之前画的)
    axprops = dict(xticks = [],yticks = [])
    createplot.ax1 = plt.subplot(111,frameon = False,**axprops) #定义绘图区，一张图，不画边框，不画刻度,**axprops逆接字典类型，即输入x=  y=  
    plottree.totalW = float(getnumleaf(tree))  #宽  定义plottree函数中plottree.totalW参数实值，下同
    plottree.totalD = float(getdepth(tree))   #高
    plottree.xOff = -0.5/plottree.totalW  #x偏移量
    plottree.yOff = 1.0   #y偏移量
    plottree(tree,(0.5,1.0),'')
    plt.show()

In [9]:
def gettree(filename):
    import pickle
    fr = open(filename,'rb') #同样地，由于二进制模式，使用rb
    tree = pickle.load(fr)  #反序列化，将其解析为一个python对象
    fr.close()
    return tree

In [20]:
finaltree = gettree('F:/毕业设计/code/ID3tree.txt')

In [21]:
createplot(finaltree)